[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/Satyajeet-code/Generative-AI/edit/main/RAG%20session%2014th%20Dec/HyDE.ipynb)


In [60]:
!pip -q install langchain faiss-cpu langchain-groq  langchain-community pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.0 MB/s eta 0:00:00


In [31]:
from bs4 import BeautifulSoup
import requests
import re
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain_core.prompts import ChatPromptTemplate
import langchain

In [4]:
groq=""

In [63]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/Applying generative AI with retrieval augmented generation to summarize and extract key clinical infor.pdf")
docs = loader.load_and_split()

In [64]:
model_name="BAAI/bge-small-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [65]:
db=FAISS.from_documents(docs,bge_embeddings)

In [66]:
llm=ChatGroq(groq_api_key=groq, model_name="Llama3-8b-8192")

In [67]:
langchain.debug=False

In [68]:
query="LLm Evaluation"
prompt = ChatPromptTemplate.from_messages([
  ("system", "Please answer the query asked by the user. Keep it short and detailed."),
  ("user", f"Question:{query}")
])

chain=prompt|llm
print("------------------------- Answer ---------------------")
response=chain.invoke({'query':query})
# print(response)
response=response.content.replace("*","\n")
print(response)

------------------------- Answer ---------------------
LLM (Large Language Model) Evaluation typically involves assessing its performance on various tasks, such as:

1. Language Understanding: Measuring the model's ability to comprehend natural language, including sentiment analysis, entity recognition, and question answering.
2. Language Generation: Evaluating the model's ability to generate coherent and relevant text, such as text summarization, language translation, and chatbot responses.
3. Common Sense and Reasoning: Testing the model's ability to apply common sense and reason, such as solving puzzles, making inferences, and understanding cause-and-effect relationships.
4. Dialogue and Conversational Flow: Assessing the model's ability to engage in natural-sounding conversations, including turn-taking, tone, and nuance.
5. Error Analysis: Identifying and analyzing errors, such as language-specific quirks, cultural biases, and domain-specific knowledge gaps.

Some popular evaluatio

In [69]:
retriever=db.as_retriever()

In [70]:
similar_docs=retriever.get_relevant_documents(response)

In [71]:
similar_docs[0].page_content

'rating RAG can mitigate hallucination by providing access to more data, \nincluding structured data. However, this approach doesn’t fundamen -\ntally solve the underlying issue within the generative model. When ac -\ncess to external data was revoked, the model reverted to incorrect \noutputs, indicating a reliance on that information. Another concern is \ninconsistency, as the model fails to produce consistent output when \npresented with minor variations in the prompt. This observation is \nconsistent with what others have reported [49,56]. Moreover, even with \nthe same prompt, the model’s output varies from one note to another \nbased on the information provided in each note. However, we found that \ngiving the model a template to follow mitigated this issue. Alternative \nprompting methods such as few-shot learning, chain of thoughts or fine- \ntuning may address zero-shot learning limitations. These methods allow \nthe model to learn from a few examples and refine its understand

In [72]:
retriever.get_relevant_documents(query)[0].page_content

'2.3. Retrieval augmented generation \nTo enable the model to access an external knowledge base including \nmore nursing notes and structured data, we utilized the RAG approach. \nThis approach involves taking nursing notes and breaking them into \nmanageable chunks of a fixed size (600 characters). This step is essential \nto facilitate subsequent processing and analysis. Similarly, when dealing \nwith structured data, this approach parses the information row by row. \nNext we utilized the sentence transformer model from Huggingface li -\nbrary [43] . This model is designed to encode sentences into dense vec -\ntors, capturing semantic meaning and relationships between words. By \nembedding the segmented data, we transform information into a nu -\nmerical representation that preserves the contextual information and \nrelationships within the data. The embedded vectors are then stored in a \nvector store along with their metadata, providing an efficient approach \nfor retrieval and ana